# Netcdf to Geotif
- Main goal is to extract and process Copernicus utci and mrt data
- Data Source : https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview
- We have a bunch of files we want to transform from Netcdf to geotif
- later we are going to do some operations as a raster calculator
- Notes: 
   - the netcdf files are in diferent folders
   - the netcdf has multiple variables with multiple dimensions
   - our netcdf has only one band


In [20]:
import netCDF4 as nc
from osgeo import gdal
import numpy as np
import os

In [21]:
"""Function to get the dates of the netcdf"""
def datelist(dimension):
    date_list = []

    for dim in ds.dimensions.values():
        if dim == ds.dimensions[dimension]:
            number_bands = len(dim) #number of bands
            variable = ds.variables[dimension]
            starting_date = variable.units
            array = variable[:]
            time_format = nc.num2date(array,starting_date)
            for date_element in time_format:
                # replace "-" for " " so "split" can work, filter and create a list by digits and finally join them in a single string with "_" as separator
                date_element_formated = "_".join([str(s) for s in str(date_element).replace(":"," ").split(" ") if s.isdigit()])

                date_list.append(date_element_formated)
        else:
            continue
        return date_list #, number_bands


In [23]:
# try: fn.close()  # just to be safe, make sure dataset is not already open.
# except: pass

gdal.UseExceptions()

folder = r"Z:\z_resources\ruben\utci_2021"
output_folder = r"Z:\z_resources\ruben\utci_2021\raster_output"
for file in os.listdir(folder):
    if file.endswith(".nc"):
        fn = os.path.join(folder, file).replace("/","\\")#.replace("\\","/")
    else:
        pass

    ds = nc.Dataset(fn) # If the file is open for write access (mode='w', 'r+' or 'a')

    dimension = "time"
    band = 0
    file_name = os.path.basename(fn)
    for t in datelist(dimension):
        band += 1
        output = os.path.join(output_folder, file_name).replace(".nc","")  + "_" + t + ".tif"
        gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(band) + " -co COMPRESS=DEFLATE -co TILED=YES NETCDF:" + str(fn) + ":utci " + str(output)
        os.system(gdaltranString)
        output = None


c:\Users\admin\.conda\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
c:\Users\admin\.conda\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


In [19]:
print(os.getcwd())

c:\Users\admin\Documents\01_Ruben_Scripts


In [5]:
"""Dimensions check"""
#each variable has its own dimensions, we want MEDIAN
for dim in ds.dimensions.values():
    print(dim) #it is no possible to iterate the dimension

"""Variables check"""
print (ds.variables.keys())

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 24

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 601

odict_keys(['time', 'lon', 'lat', 'utci'])


c:\Users\admin\.conda\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  after removing the cwd from sys.path.


In [18]:
utci_variable = ds.variables['utci']
print(utci_variable.dimensions)

('time', 'lat', 'lon')


In [43]:
"""Here we start transforming the data"""
gdal.UseExceptions()

file_name = os.path.basename(fn)
output = os.path.join(main_path, file_name).replace(".nc",".tif")
band = 1

# nn = gdal.Translate(output, fn, 
#                 format='GTIFF',
#                 # format='NetCDF',
#                 # bandList = [1]
#                 # creationOptions = ['COMPRESS=DEFLATE','TILED=YES']
#                 )

gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(band) + " -co COMPRESS=DEFLATE -co TILED=YES NETCDF:" + str(fn) + ":utci " + str(output)
# gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(1) + " -co COMPRESS=DEFLATE -co TILED=YES " + str(fn) + " " + str(output)

os.system(gdaltranString)
output = None

#gdal_translate -of GTiff -b 10 input.nc output.tiff  # to get 10th band, it works fine

https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview